# A5: Natural Language Inference using Neural Networks

Adam Ek

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Write all your answers and the code in the appropriate boxes below.


In this lab we will work with neural networks for natural language inference. Our task is: given a premise sentence P and hypothesis H, what entailment relationship holds between them? Is H entailed by P, contradicted by P or neutral towards P?

Given a sentence P, if H definitely describe something true given P then it is an **entailment**. If H describe something that's *maybe* true given P, it's **neutral**, and if H describe something that's definitely *false* given P it's a **contradiction**.

In [20]:
import math
import sys
from pprint import pprint

import numpy as np
import torch
import torch.nn as nn
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score, precision_score, recall_score
from torch import optim
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, Dataset

In [21]:
PADDING_TOKEN = '<PAD>'
UNKNOWN_TOKEN = '<UNK>'
UNKNOWN_LABEL = '<UNK>'

device = torch.device('cuda:0')

In [22]:
hyperparameters = {
    'batch_size': 128,
    'embedding_dim': 512,
    'lstm_out_dim': 512,
    'epochs': 3,
    'learning_rate': 0.002
}

## 1. Data

We will explore natural language inference using neural networks on the SNLI dataset, described in [1]. The dataset can be downloaded [here](https://nlp.stanford.edu/projects/snli/). We prepared a "simplified" version, with only the relevant columns in `simple_snli_1.0.zip`.

The (simplified) data is organized as follows (tab-separated values):
* Column 1: Premise
* Column 2: Hypothesis
* Column 3: Relation

Like in the previous lab, we'll use torchtext to build a dataloader. You can essentially do the same thing as you did in the last lab, but with our new dataset. **[1 mark]**

In [23]:
class SNLIDataset(Dataset):
    def __init__(self, path, dataset=None, only_consensus=True):
        data_file = self._read_file(path, only_consensus)

        if dataset is None:
            self.max_length_context = -1
            vocab = {PADDING_TOKEN, UNKNOWN_TOKEN}
            relations = {UNKNOWN_LABEL}
            for sample in data_file:
                vocab.update(sample['premise'], sample['hypothesis'])
                relations.add(sample['relation'])
                self.max_length_context = max(self.max_length_context, len(sample['premise']),
                                              len(sample['hypothesis']))

            self.vocab = {word: index for index, word in enumerate(list(vocab))}
            self.relations = {relation: index for index, relation in enumerate(list(relations))}
        else:
            self.vocab = dataset.vocab
            self.relations = dataset.relations
            self.max_length_context = dataset.max_length_context

        self.samples = []
        for sample in data_file:
            premise = [self.get_encoded_word(word) for word in sample['premise']]
            premise.extend([self.get_encoded_word(PADDING_TOKEN)] * (self.max_length_context - len(sample['premise'])))

            hypothesis = [self.get_encoded_word(word) for word in sample['hypothesis']]
            hypothesis.extend(
                [self.get_encoded_word(PADDING_TOKEN)] * (self.max_length_context - len(sample['hypothesis'])))

            self.samples.append({
                'premise': torch.tensor(premise),
                'hypothesis': torch.tensor(hypothesis),
                'relation': torch.tensor(self.get_encoded_relation(sample['relation']))
            })

    def _read_file(self, path, only_consensus):
        NO_CONSENSUS_RELATION = '-'
        lines = []
        with open(path) as f:
            for sample in f:
                premise, hypothesis, relation = sample.rstrip().split('\t')
                if relation != NO_CONSENSUS_RELATION or not only_consensus:
                    lines.append({
                        'premise': [word.lower() for word in word_tokenize(premise)],
                        'hypothesis': [word.lower() for word in word_tokenize(hypothesis)],
                        'relation': relation
                    })
        return lines

    def get_encoded_word(self, word):
        if word in self.vocab:
            return self.vocab[word]
        else:
            return self.vocab[UNKNOWN_TOKEN]

    def get_encoded_relation(self, label):
        if label in self.relations:
            return self.relations[label]
        else:
            return self.relations[UNKNOWN_LABEL]

    def __getitem__(self, item):
        return self.samples[item]

    def __len__(self):
        return len(self.samples)

In [24]:
dataset = SNLIDataset('simple_snli_1.0_dev.csv')
print(dataset[:2])

[{'premise': tensor([5400, 4646, 2774,  194, 5420, 4213, 2711, 1420, 3911, 4639, 1120, 1120,
        1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120,
        1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120,
        1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120,
        1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120]), 'hypothesis': tensor([4824,  439, 2774, 2274,  969, 5420, 4213, 2711, 1420, 3911, 2911, 2531,
        3779, 4202, 4639, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120,
        1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120,
        1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120,
        1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120]), 'relation': tensor(0)}, {'premise': tensor([5400, 4646, 2774,  194, 5420, 4213, 2711, 1420, 3911, 4639, 1120, 1120,
        1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 112

In [25]:
def dataloader(path_train, path_test, batch_size):
    train_dataset = SNLIDataset(path_train)
    test_dataset = SNLIDataset(path_test, train_dataset)

    train_dataloader = DataLoader(train_dataset,
                                  batch_size=batch_size,
                                  shuffle=True)
    test_dataloader = DataLoader(test_dataset,
                                 batch_size=batch_size,
                                 shuffle=True)

    return train_dataloader, test_dataloader

## 2. Model

In this part, we'll build the model for predicting the relationship between H and P.

We will process each sentence using an LSTM. Then, we will construct some representation of the sentence. When we have a representation for H and P, we will combine them into one vector which we can use to predict the relationship.

We will train a model described in [2], the BiLSTM with max-pooling model. The procedure for the model is roughly:

    1) Encode the Hypothesis and the Premise using one shared bidirectional LSTM (or two different LSTMS)
    2) Perform max over the tokens in the premise and the hypothesis
    3) Combine the encoded premise and encoded hypothesis into one representation
    4) Predict the relationship 

### Creating a representation of a sentence

Let's first consider step 2 where we perform max/mean pooling. There is a function in pytorch for this, but we'll implement it from scratch. 

Let's consider the general case, what we want to do for these methods is apply some function $f$ along dimension $i$, and we want to do this for all $i$'s. As an example we consider the matrix S with size ``(N, D)`` where N is the number of words and D the number of dimensions:

$S = \begin{bmatrix}
    s_{11} & s_{12} & s_{13} & \dots  & s_{1d} \\
    s_{21} & s_{22} & s_{23} & \dots  & s_{2d} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    s_{n1} & s_{n2} & s_{n3} & \dots  & s_{nd}
\end{bmatrix}$

What we want to do is apply our function $f$ on each dimension, taking the input $s_{1d}, s_{2d}, ..., s_{nd}$ and generating the output $x_d$. 

You will implement both the max pooling method. When performing max-pooling, $max$ will be the function which selects a _maximum_ value from a vector and $x$ is the output, thus for each dimension $d$ in our output $x$ we get:

\begin{equation}
    x_d = max(s_{1d}, s_{2d}, ..., s_{nd})
\end{equation}


This operation will reduce a batch of size ``(batch_size, num_words, dimensions)`` to ``(batch_size, dimensions)`` meaning that we now have created a sentence representation based on the content of the words representations in the sentence. 

Create a function that takes as input a tensor of size ``(batch_size, num_words, dimensions)`` then performs max pooling and returns the result (the output should be of size: ```(batch_size, dimensions)```). [**4 Marks**]

In [26]:
def pooling(input_tensor, method='max'):
    output_tensor = []
    for sample in input_tensor:
        transposed = torch.transpose(sample, 0, 1)

        dim = []
        for dimension in transposed:
            if method == 'max':
                dim.append(max(dimension))
            elif method == 'mean':
                dim.append(sum(dimension) / len(dimension))
            else:
                raise ValueError("only 'max' or 'mean' pooling possible")

        output_tensor.append(dim)
    return torch.tensor(output_tensor)

In [27]:
def pooling_fast(input_tensor, method='max'):
    if method == 'max':
        return torch.max(input_tensor, dim=1).values
    elif method == 'mean':
        return torch.mean(input_tensor, dim=1).values
    else:
        raise ValueError("only 'max' or 'mean' pooling possible")

In [28]:
t1 = torch.randn(16, 59, 256)

slow = pooling(t1)
fast = pooling_fast(t1)
print(torch.equal(slow, fast))

True


### Combining sentence representations

Next, we need to combine the premise and hypothesis into one representation. We will do this by concatenating four tensors (the final size of our tensor $X$ should be ``(batch_size, 4d)`` where ``d`` is the number of dimensions that you use): 

$$X = [P; H; |P-H|; P \cdot H]$$

Here, what we do is concatenating P, H, P times H, and the absolute value of P minus H, then return the result.

Implement the function. **[2 marks]**

In [29]:
def combine_premise_and_hypothesis(premise, hypothesis):
    return torch.cat((
        premise,
        hypothesis,
        premise - hypothesis,
        premise * hypothesis
    ), 1)

### Creating the model

Finally, we can build the model according to the procedure given previously by using the functions we defined above. Additionaly, in the model you should use *dropout*. For efficiency purposes, it's acceptable to only train the model with either max or mean pooling. 

Implement the model [**6 marks**]

In [30]:
class SNLIModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, lstm_out_dim, number_of_relations, padding_idx):
        super(SNLIModel, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.rnn = nn.LSTM(embedding_dim, lstm_out_dim, bidirectional=True, batch_first=True)
        self.classifier = nn.Linear(2 * 4 * lstm_out_dim, number_of_relations)
        self.dropout = nn.Dropout(p=0.25)

    def forward(self, premise, hypothesis):
        p_embedding = self.embeddings(premise)
        h_embedding = self.embeddings(hypothesis)

        p_out, _ = self.rnn(p_embedding)
        h_out, _ = self.rnn(h_embedding)

        p_out = self.dropout(p_out)
        h_out = self.dropout(h_out)

        p_pooled = pooling_fast(p_out)
        h_pooled = pooling_fast(h_out)

        ph_representation = combine_premise_and_hypothesis(p_pooled, h_pooled)

        predictions = self.classifier(ph_representation)

        return predictions

## 3. Training and testing

As before, implement the training and testing of the model. SNLI can take a very long time to train, so I suggest you only run it for one or two epochs. **[2 marks]** 

**Tip for efficiency:** *when developing your model, try training and testing the model on one batch (for each epoch) of data to make sure everything works! It's very annoying if you train for N epochs to find out that something went wrong when testing the model, or to find that something goes wrong when moving from epoch 0 to epoch 1.*

In [31]:
train_dataloader, test_dataloader = dataloader('simple_snli_1.0_train.csv',
                                               'simple_snli_1.0_test.csv',
                                               hyperparameters['batch_size'])
train_dataset = train_dataloader.dataset

In [32]:
loss_function = CrossEntropyLoss()
snli_model = SNLIModel(len(train_dataset.vocab),
                       hyperparameters['embedding_dim'],
                       hyperparameters['lstm_out_dim'],
                       len(train_dataset.relations),
                       train_dataset.get_encoded_word(PADDING_TOKEN))
snli_model.to(device)

optimizer = optim.Adam(snli_model.parameters(), lr=hyperparameters['learning_rate'])

In [33]:
pprint(hyperparameters)
print(f"vocab_size: {len(train_dataset.vocab)}")
print(f"number relations: {len(train_dataset.relations)}")
print(f"sentence length: {train_dataset.max_length_context}")

{'batch_size': 128,
 'embedding_dim': 512,
 'epochs': 3,
 'learning_rate': 0.002,
 'lstm_out_dim': 512}
vocab_size: 36548
number relations: 4
sentence length: 82


In [34]:
print(
    f'{hyperparameters["epochs"]} EPOCHS - {math.floor(len(train_dataset) / train_dataloader.batch_size)} BATCHES PER EPOCH')

for epoch in range(hyperparameters['epochs']):
    total_loss = 0
    for i, batch in enumerate(train_dataloader):
        batch_premise = batch['premise'].to(device)
        batch_hypothesis = batch['hypothesis'].to(device)
        batch_relation = batch['relation'].to(device)

        output = snli_model(batch_premise, batch_hypothesis)

        loss = loss_function(output.view(-1, len(dataset.relations)), batch_relation.view(-1))
        total_loss += loss.item()

        # print average loss for the epoch
        sys.stdout.write(f'\repoch {epoch}, batch {i}: {np.round(total_loss / (i + 1), 4)}')

        # compute gradients
        loss.backward()

        # update parameters
        optimizer.step()

        # reset gradients
        optimizer.zero_grad()
    print()

3 EPOCHS - 4291 BATCHES PER EPOCH
epoch 0, batch 4291: 0.7172
epoch 1, batch 4291: 0.6332
epoch 2, batch 4291: 0.6999


In [35]:
gold_relations = []
predicted_relations = []

for batch in test_dataloader:
    batch_premise = batch['premise'].to(device)
    batch_hypothesis = batch['hypothesis'].to(device)

    model_output = snli_model(batch_premise, batch_hypothesis).detach().cpu()
    predictions = torch.max(model_output, dim=1).indices
    for index, gold_relation in enumerate(batch['relation']):
        predicted_relations.append(int(predictions[index]))
        gold_relations.append(int(gold_relation))

In [36]:
print(f"""
Accuracy: {accuracy_score(gold_relations, predicted_relations):.4f}
Precision: {precision_score(gold_relations, predicted_relations, average='macro'):.4f}
Recall: {recall_score(gold_relations, predicted_relations, average='macro'):.4f}
""")


Accuracy: 0.7552
Precision: 0.7544
Recall: 0.7545



Suggest a _baseline_ that we can compare our model against **[2 marks]**

**Your answer should go here**

Suggest some ways (other than using a baseline) in which we can analyse the models performance **[4 marks]**.

**Your answer should go here**

Suggest some ways to improve the model **[3 marks]**.

**Your answer should go here**

## Readings

[1] Samuel R. Bowman, Gabor Angeli, Christopher Potts, and Christopher D. Manning. 2015. A large annotated corpus for learning natural language inference. In Proceedings of the 2015 Conference on Empirical Methods in Natural Language Processing (EMNLP). 

[2] Conneau, A., Kiela, D., Schwenk, H., Barrault, L., & Bordes, A. (2017). Supervised learning of universal sentence representations from natural language inference data. arXiv preprint arXiv:1705.02364.

## Statement of contribution

Briefly state how many times you have met for discussions, who was present, to what degree each member contributed to the discussion and the final answers you are submitting.

## Marks

This assignment has a total of 23 marks.